# Multi objective optimization post-processing analysis

In [ ]:
#import WarmupFUSE
using FUSE
using Plots

In [ ]:
path= "optimization_runs"
path= "/mnt/beegfs/users/meneghini/optimization_runs_supersimple_sms"
path= "/mnt/beegfs/users/meneghini/optimization_runs_supersimple_NSGA2"
path= "/mnt/beegfs/users/meneghini/optimization_runs_simple_NSGA2"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta"
path= "/mnt/beegfs/users/meneghini/optimization_run_STEP_nodelta_zerohm"

In [ ]:
# use this to regenerate CSV cache file
dirs = filter(isdir,sort(readdir(path; join=true)))
println(length(dirs))
dirs = filter(x->!isfile(joinpath(x,"error.txt")) && isfile(joinpath(x,"dd.json")),dirs)
println(length(dirs))
outputs=FUSE.extract(dirs, filter_invalid=:cols);

# cache extrated information to CSV file
import DelimitedFiles
DelimitedFiles.writedlm(joinpath(path,"extract.csv"), Iterators.flatten(([names(outputs)], eachrow(outputs))), ',')

# error analysis
dirs = filter(isdir,sort(readdir(path; join=true)))
errors=FUSE.categorize_errors(dirs;do_plot=true,show_first_line=true)

In [ ]:
# use this to load CSV cache file
outputs=FUSE.DataFrames.DataFrame(FUSE.CSV.File(joinpath(path,"extract.csv")));

In [ ]:
# this is just to list the fields that can be queried
IMAS.ExtractFunctionsLibrary

In [ ]:
scatter(outputs[:,"Pelectric_net"])
display(hline!([200]))

scatter(log10.(outputs[:,"flattop"]))
display(hline!([1]))

In [ ]:
for name in names(outputs)
    if all(isnan.(outputs[:,name]))
        continue
    end
    println(name)
    display(histogram(outputs[:,name],label="",size=(200,100),linecolor=nothing))
end

In [ ]:
# x axis
xname="capital_cost"
x=outputs[:,xname]

# y axis
yname="βn"
y=outputs[:,yname]

# z axis
zname="R0"
z=outputs[:,zname]

# color
cname="R0"
cname="<zeff>"
c=outputs[:,cname]
clim=(-Inf,Inf)
#cname="B0 [T]"
#cname="R0 [m]"

# selection
sname0="Pelectric_net"
s0=outputs[:,sname0]

# selection
sname1="ip_ohm"
s1=outputs[:,sname1]

# subselection criterion
n=20.0
index=1:length(x)
index=findall( (s0.>(200-n)) .&& (s0.<(200+n)) .&& (s1.>-0.5) .&& (s1.<0.5) .&& (index.>2000))

annot=map(x->(x, :center, 3, "courier"),index)

# plotting
scatter(x[index], y[index], marker_z=c[index] ,xlabel="Capital cost [\$B]", ylabel=yname, colorbar_title=cname, marker=:circle,
    markersize=5, markerstrokewidth=0, label="", clim=clim, alpha=0.5,
#    xlim=(0,Inf), ylim=(0,Inf))
    xlim=(3,15), ylim=(1,3.5))

# scatter3d(x[index], y[index], z[index], marker_z=c[index] ,xlabel="Capital cost [\$B]", ylabel=yname, colorbar_title=cname, marker=:circle,
#     markersize=5, markerstrokewidth=0, label="", clim=clim, alpha=0.5,
#     xlim=(0,Inf), ylim=(0,Inf))
# #    xlim=(3,15), ylim=(1,3.5))

#    xlim=(3,10), ylim=(1.0,3.0), series_annotations=annot, textfontsize=2)
# for i in 1:length(x)
#     ann = (x[i], y[i], "$i")
#     annotate!(ann, text_color=:black, series_annotations=true, alpha=.8, valign=:bottom)
# end

In [ ]:
n=4901
#n=7864
println(dirs[n])
dd,ini,act=FUSE.load(dirs[n])
FUSE.digest(dd)
#ini.core_profiles

In [ ]:
for cname in names(outputs)
    if all(isnan.(outputs[:,cname]))
        continue
    end
    # color
    c=outputs[:,cname]
    clim=(-Inf,Inf)

    # plotting
    println(cname)
    display(scatter(x[index], y[index], marker_z=c[index] ,xlabel=xname, ylabel=yname, colorbar_title=cname, marker=:circle,
        markersize=5, markerstrokewidth=0, label="", clim=clim, alpha=0.5))
end